In [1]:
import os
from dotenv import load_dotenv

from src.agents.graph_qa import GraphAgentResponder
from src.config import LLMConf, EmbedderConf, KnowledgeGraphConfig
from src.graph.knowledge_graph import KnowledgeGraph
from src.ingestion.embedder import ChunkEmbedder


env = load_dotenv('config.env')

In [2]:
kg_config = KnowledgeGraphConfig(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name=os.getenv("INDEX_NAME")
)

# llm_conf = LLMConf(
#     type="ollama",
#     model="llama3.2:latest", 
#     temperature=0.0, 
# )

llm_conf=LLMConf(
    type=os.getenv("QA_MODEL_TYPE"),
    model=os.getenv("QA_MODEL_NAME"), 
    temperature=os.getenv("QA_MODEL_TEMPERATURE"), 
    deployment=os.getenv("QA_MODEL_DEPLOYMENT"),
    api_key=os.getenv("QA_API_KEY"),
    endpoint=os.getenv("QA_MODEL_ENDPOINT"),
    api_version=os.getenv("QA_MODEL_API_VERSION") or None
)

embedder_conf = EmbedderConf(
    type="ollama",
    model="mxbai-embed-large",
)

In [3]:
embedder = ChunkEmbedder(conf=embedder_conf)

knowledge_graph = KnowledgeGraph(
    conf=kg_config, 
    embeddings_model=embedder.embeddings
)

knowledge_graph._driver.verify_connectivity()

knowledge_graph._driver.verify_authentication()

True

In [4]:
knowledge_graph.get_structured_schema # too long to pass in prompt

{'node_props': {'Chunk': [{'property': 'id', 'type': 'STRING'},
   {'property': 'embedding', 'type': 'LIST'},
   {'property': 'text', 'type': 'STRING'},
   {'property': 'chunk_size', 'type': 'INTEGER'},
   {'property': 'filename', 'type': 'STRING'},
   {'property': 'document_version', 'type': 'INTEGER'},
   {'property': 'chunk_overlap', 'type': 'INTEGER'},
   {'property': 'chunk_id', 'type': 'INTEGER'},
   {'property': 'embeddings_model', 'type': 'STRING'},
   {'property': 'folder', 'type': 'STRING'}],
  'Document': [{'property': 'filename', 'type': 'STRING'},
   {'property': 'document_version', 'type': 'INTEGER'}],
  'Person': [{'property': 'id', 'type': 'STRING'},
   {'property': 'name', 'type': 'STRING'},
   {'property': 'description', 'type': 'STRING'},
   {'property': 'title', 'type': 'STRING'},
   {'property': 'type', 'type': 'STRING'}],
  'Country': [{'property': 'id', 'type': 'STRING'},
   {'property': 'name', 'type': 'STRING'},
   {'property': 'description', 'type': 'STRING'},

In [5]:
knowledge_graph.labels

['Chunk',
 'Document',
 'Person',
 'Country',
 '__Entity__',
 'Organization',
 'European union',
 'Event',
 'Agreement',
 'Financial instrument',
 'Infrastructure',
 'Expenses',
 'Mechanism',
 'Facility',
 'Financial institution',
 'Central securities depositories',
 'Central bank of russia',
 'Ukraine',
 'Assets',
 'Reserves',
 'Revenues',
 'Financial mechanism',
 'Geopolitical entity']

In [6]:
knowledge_graph.relationships

['MENTIONS',
 'PART_OF',
 'NEXT',
 'SUPPORTS',
 'CONTRIBUTES',
 'FUNDS',
 'REGULATES',
 'OWNERSHIP',
 'HOLDS',
 'GENERATES',
 'RECEIVES',
 'CONTRIBUTES_TO',
 'SUPPORT',
 'DISBURSES',
 'REPAID_WITH',
 'RECIPIENT',
 'AGGRESSOR',
 'DESTROYER',
 'PROVIDER',
 'REPAYMENT_MECHANISM',
 'BENEFICIARY',
 'PROVIDES_SUPPORT',
 'DISBURSES_LOAN',
 'WAGES_WAR',
 'AIDS',
 'SANCTION',
 'AGGRESSION',
 'EFFECT',
 'IMPOSITION',
 'DECIDES',
 'BENEFITS',
 'DECIDED_TO_SUPPORT',
 'PROVIDED_FINANCIAL_AID',
 'AIMS_TO_SUPPORT',
 'WILL_BE_USED_TO',
 'PARTNERED_WITH',
 'OWNED_BY',
 'IMMOBILIZED_ASSETS_OF',
 'PROVIDES',
 'ATTACKS',
 'COMMENTS_ON',
 'WORKS_FOR',
 'RECEIVES_LOAN']

In [7]:
responder = GraphAgentResponder(
    qa_llm_conf=llm_conf, # TODO try different LLMs
    cypher_llm_conf=llm_conf,
    graph=knowledge_graph,
    rephrase_llm_conf=llm_conf
)

In [8]:
query = "What countries are mentioned in the graph?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...


'Russia and Ukraine.'

In [9]:
query = "Is Russia mentioned in the graph?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...


'Yes, Russia is mentioned in the context.'

In [10]:
query = "What document mentions Ukraine?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...


'The document that mentions Ukraine is the European Commission Press release, specifically "Commission disburses first €3 billion to Ukraine of its part of the G7 loan, to be repaid with proceeds from immobilised Russian assets".'

In [11]:
query = "When was the request for payment from Germany submitted?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...


"There is no information available about a request for payment from Germany. The provided context and query result do not mention a request for payment from Germany, but rather discuss the European Commission's disbursement of €3 billion to Ukraine as part of the G7 loan, which will be repaid with proceeds from immobilised Russian State assets in the EU."

In [12]:
query = "What happened in the baltic sea?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...


'There is no information about the Baltic Sea in the provided context or query result.'